In [144]:
import pandas as pd
from datetime import datetime, timedelta
from scipy import stats
import numpy as np

In [217]:
import sqlite3
conn = sqlite3.connect('production.db') 
c=conn.cursor()

In [146]:
raw_data = pd.read_sql_query("SELECT * from raw_data", conn)

In [147]:
conn.close()

In [148]:
raw_data

,area,date_hour,biomass,fossil_brown_coal_lignite,fossil_coal_derived_gas,fossil_gas,fossil_hard_coal,fossil_oil,fossil_oil_shale,fossil_peat,...,hydro_run_river_poundage,hydro_water_reservoir,marine,nuclear,other,other_renewable,solar,waste,wind_offshore,wind_onshore
0,BZN|DK2,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,n/e,n/e,41.0,198.0,4.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,72.0,119.0,255.0
1,BZN|DK2,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,n/e,n/e,41.0,187.0,4.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,73.0,105.0,219.0
2,BZN|DK2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,n/e,n/e,34.0,184.0,4.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,72.0,96.0,226.0
3,BZN|DK2,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,n/e,n/e,32.0,180.0,4.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,73.0,114.0,239.0
4,BZN|DK2,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,n/e,n/e,32.0,184.0,8.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,70.0,137.0,254.0
5,BZN|DK2,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,n/e,n/e,32.0,186.0,12.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,144.0,302.0
6,BZN|DK2,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,n/e,n/e,32.0,185.0,12.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,63.0,168.0,329.0
7,BZN|DK2,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,n/e,n/e,33.0,188.0,12.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,168.0,346.0
8,BZN|DK2,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,n/e,n/e,34.0,187.0,12.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,149.0,309.0
9,BZN|DK2,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,n/e,n/e,34.0,165.0,13.0,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,6.0,65.0,130.0,316.0


# Following steps were perfermed on the data set for data pre-procesing and cleaning: 
1. Dropping columns with no data points or redundancy.
2. Deriving meaningful columns such as Month,Time,WeekName, WeekNumber out of date_Hour column 
3. Handling missing values and replacing them with the mean of the respective column.
4. Missing value analysis
5. Outlier Detection 
6. Outlier Analysis

In [149]:
raw_data['area'].unique() #dropping Area columns since I am already aware of the electricity zone I am performing my analysis on. 

array(['BZN|DK2'], dtype=object)

In [150]:
raw_data['area'].nunique()

1

In [151]:
cleaned_data=raw_data.drop(['area'], axis = 1) # area column carries redundant values

In [152]:
cleaned_data

,date_hour,biomass,fossil_brown_coal_lignite,fossil_coal_derived_gas,fossil_gas,fossil_hard_coal,fossil_oil,fossil_oil_shale,fossil_peat,geothermal,...,hydro_run_river_poundage,hydro_water_reservoir,marine,nuclear,other,other_renewable,solar,waste,wind_offshore,wind_onshore
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,n/e,n/e,41.0,198.0,4.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,72.0,119.0,255.0
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,n/e,n/e,41.0,187.0,4.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,73.0,105.0,219.0
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,n/e,n/e,34.0,184.0,4.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,72.0,96.0,226.0
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,n/e,n/e,32.0,180.0,4.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,73.0,114.0,239.0
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,n/e,n/e,32.0,184.0,8.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,70.0,137.0,254.0
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,n/e,n/e,32.0,186.0,12.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,144.0,302.0
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,n/e,n/e,32.0,185.0,12.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,63.0,168.0,329.0
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,n/e,n/e,33.0,188.0,12.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,168.0,346.0
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,n/e,n/e,34.0,187.0,12.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,0.0,64.0,149.0,309.0
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,n/e,n/e,34.0,165.0,13.0,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,6.0,65.0,130.0,316.0


# Dropping columns with n/e values
These columns are ireelavnt and carries no data point. Keeping irrelevant columns in the dataframe will make the data very chaotic

In [153]:
cleaned_data=cleaned_data.drop(['fossil_brown_coal_lignite', 'fossil_coal_derived_gas', 
           'fossil_oil_shale','fossil_peat','geothermal','hydro_pumped_storage',
          'hydro_pumped_storage_consumption','hydro_run_river_poundage',
          'hydro_water_reservoir','marine','nuclear','other','other_renewable'], axis = 1)

In [154]:
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0


In [155]:
cleaned_data.shape

(8785, 9)

In [156]:
cleaned_data.dtypes

date_hour            object
biomass             float64
fossil_gas          float64
fossil_hard_coal    float64
fossil_oil          float64
solar               float64
waste               float64
wind_offshore       float64
wind_onshore        float64
dtype: object

# Fixing the date_hour column
1. The date is in string. Hence I am going to convert it into date time stamp 
2. I am also going to carry out feature engineering to derive attributes such as hour, weekday, 
week number, months out of Date Time 

In [157]:
cleaned_data['date'] = cleaned_data['date_hour'].str.slice(0, 10)

In [158]:
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,01.01.2020
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,01.01.2020
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,01.01.2020
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,01.01.2020
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,01.01.2020
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,01.01.2020
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,01.01.2020
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,01.01.2020
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,01.01.2020
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,01.01.2020


In [159]:
cleaned_data['date']=cleaned_data['date'].str.replace('.','/')
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,01/01/2020
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,01/01/2020
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,01/01/2020
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,01/01/2020
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,01/01/2020
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,01/01/2020
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,01/01/2020
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,01/01/2020
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,01/01/2020
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,01/01/2020


In [160]:
cleaned_data['date']=cleaned_data['date'].apply(lambda _: datetime.strptime(_,"%d/%m/%Y"))
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01


In [161]:
cleaned_data.dtypes

date_hour                   object
biomass                    float64
fossil_gas                 float64
fossil_hard_coal           float64
fossil_oil                 float64
solar                      float64
waste                      float64
wind_offshore              float64
wind_onshore               float64
date                datetime64[ns]
dtype: object

# Deriving meaningful attributes out of Date Time column

In [162]:
cleaned_data['weekday'] = cleaned_data['date'].dt.day_name() #weekday
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday


In [163]:
cleaned_data['week_number'] = cleaned_data['date'].dt.week #weeknumber
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1


In [164]:
def hr_func(ts):
    return int(ts[30:32])

cleaned_data['hour'] = cleaned_data['date_hour'].apply(hr_func) #hour 
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10


In [165]:
cleaned_data['month'] = cleaned_data['date'].dt.month_name() #month
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1,January
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2,January
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3,January
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4,January
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5,January
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6,January
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7,January
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8,January
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9,January
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10,January


# Handling missing values

In [166]:
cleaned_data.isnull().sum()

date_hour           0
biomass             1
fossil_gas          1
fossil_hard_coal    1
fossil_oil          1
solar               1
waste               1
wind_offshore       1
wind_onshore        1
date                0
weekday             0
week_number         0
hour                0
month               0
dtype: int64

In [167]:
cleaned_data.describe()

,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,week_number,hour
count,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8784.000000,8785.000000,8785.000000
mean,201.599954,49.728825,92.256148,16.630464,45.605305,60.306808,148.926571,225.275387,26.930791,11.499032
std,161.944944,33.533356,73.367649,16.024015,73.864572,13.363553,123.638688,180.079200,15.099209,6.922781
min,1.000000,11.000000,17.000000,4.000000,0.000000,15.000000,0.000000,13.000000,1.000000,0.000000
25%,18.000000,29.000000,45.000000,9.000000,0.000000,52.000000,38.000000,73.000000,14.000000,5.000000
50%,194.000000,43.000000,70.000000,14.000000,1.000000,60.000000,119.000000,171.000000,27.000000,11.000000
75%,316.000000,57.000000,128.000000,21.000000,64.000000,71.000000,256.000000,356.000000,40.000000,17.000000
max,662.000000,889.000000,3599.000000,304.000000,311.000000,146.000000,377.000000,724.000000,53.000000,23.000000


In [168]:
cleaned_data = cleaned_data.fillna(value=cleaned_data.mean())
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1,January
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2,January
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3,January
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4,January
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5,January
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6,January
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7,January
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8,January
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9,January
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10,January


In [169]:
cleaned_data.describe()

,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,week_number,hour
count,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000
mean,201.599954,49.728825,92.256148,16.630464,45.605305,60.306808,148.926571,225.275387,26.930791,11.499032
std,161.935726,33.531447,73.363473,16.023102,73.860368,13.362793,123.631650,180.068950,15.099209,6.922781
min,1.000000,11.000000,17.000000,4.000000,0.000000,15.000000,0.000000,13.000000,1.000000,0.000000
25%,18.000000,29.000000,45.000000,9.000000,0.000000,52.000000,38.000000,73.000000,14.000000,5.000000
50%,194.000000,43.000000,70.000000,14.000000,1.000000,60.000000,119.000000,171.000000,27.000000,11.000000
75%,316.000000,57.000000,128.000000,21.000000,64.000000,71.000000,256.000000,356.000000,40.000000,17.000000
max,662.000000,889.000000,3599.000000,304.000000,311.000000,146.000000,377.000000,724.000000,53.000000,23.000000


#### From the above results it is evident that replacing the respective missing values with the mean of each respective column did not impact the mean or standard deviation. Hence missing values have been successfully handled! 

In [170]:
cleaned_data['biomass'].unique()

array([432.        , 460.        , 413.        , 411.        ,
       458.        , 409.        , 410.        , 479.        ,
       456.        , 466.        , 461.        , 465.        ,
       470.        , 468.        , 425.        , 424.        ,
       422.        , 412.        , 414.        , 390.        ,
       399.        , 367.        , 320.        , 336.        ,
       375.        , 457.        , 504.        , 518.        ,
       525.        , 526.        , 524.        , 510.        ,
       502.        , 496.        , 490.        , 497.        ,
       441.        , 442.        , 397.        , 271.        ,
       225.        , 228.        , 230.        , 288.        ,
       255.        , 449.        , 492.        , 489.        ,
       491.        , 487.        , 471.        , 493.        ,
       494.        , 486.        , 439.        , 312.        ,
       275.        , 241.        , 242.        , 268.        ,
       265.        , 256.        , 266.        , 276.  

In [171]:
cleaned_data = cleaned_data.round(decimals = 0)

In [172]:
cleaned_data['biomass'].unique()

array([432., 460., 413., 411., 458., 409., 410., 479., 456., 466., 461.,
       465., 470., 468., 425., 424., 422., 412., 414., 390., 399., 367.,
       320., 336., 375., 457., 504., 518., 525., 526., 524., 510., 502.,
       496., 490., 497., 441., 442., 397., 271., 225., 228., 230., 288.,
       255., 449., 492., 489., 491., 487., 471., 493., 494., 486., 439.,
       312., 275., 241., 242., 268., 265., 256., 266., 276., 381., 478.,
       483., 481., 384., 426., 501., 506., 505., 508., 507., 498., 462.,
       443., 440., 419., 451., 463., 503., 499., 509., 472., 469., 361.,
       376., 373., 387., 453., 435., 420., 488., 459., 522., 428., 423.,
       514., 448., 405., 495., 467., 521., 538., 545., 542., 523., 403.,
       263., 248., 249., 252., 247., 269., 324., 352., 335., 332., 398.,
       388., 430., 438., 407., 389., 366., 393., 482., 515., 243., 281.,
       400., 434., 530., 531., 386., 383., 358., 360., 569., 632., 610.,
       615., 613., 561., 429., 475., 476., 455., 43

In [173]:
cleaned_data.isnull().sum()

date_hour           0
biomass             0
fossil_gas          0
fossil_hard_coal    0
fossil_oil          0
solar               0
waste               0
wind_offshore       0
wind_onshore        0
date                0
weekday             0
week_number         0
hour                0
month               0
dtype: int64

# Handling outliers
I will be calculating Z score to find existance of any outlier present in the dataset. 

In [174]:
z_biomass = np.abs(stats.zscore(cleaned_data['biomass']))
print(z_biomass)

[1.42286771 1.59578566 1.30553053 ... 2.10218822 2.13924207 2.07131002]


In [175]:
threshold = 3
print(np.where(z_biomass > 3)) #no outliers detected in biomass column

(array([], dtype=int64),)


In [176]:
z_fossil_gas=np.abs(stats.zscore(cleaned_data['fossil_gas']))
print(z_fossil_gas)

[0.26033332 0.26033332 0.46910446 ... 1.11159132 0.78352239 0.93264463]


In [177]:
threshold_f = 3
tup_fg = np.where(z_fossil_gas > 3)

In [178]:
len(tup_fg[0]) #168 outliers detected in fossil_gas

168

In [179]:
lis_fg = list(tup_fg[0]) #converting the tuples into list to calculate the number of outliers
lis_fg

[201,
 522,
 848,
 1064,
 1065,
 1073,
 1352,
 1353,
 1378,
 1379,
 1380,
 1383,
 1384,
 1385,
 1386,
 1474,
 1475,
 1476,
 1507,
 1529,
 1530,
 1531,
 1574,
 1578,
 1579,
 1580,
 1645,
 1646,
 6211,
 6946,
 7017,
 7595,
 7596,
 7597,
 7649,
 7650,
 7651,
 7652,
 7721,
 7790,
 7796,
 7857,
 7915,
 7984,
 8008,
 8025,
 8026,
 8027,
 8028,
 8029,
 8030,
 8031,
 8032,
 8033,
 8034,
 8035,
 8036,
 8037,
 8038,
 8041,
 8059,
 8073,
 8074,
 8075,
 8076,
 8077,
 8078,
 8079,
 8080,
 8081,
 8082,
 8083,
 8084,
 8085,
 8128,
 8129,
 8130,
 8131,
 8133,
 8134,
 8217,
 8218,
 8221,
 8222,
 8223,
 8224,
 8225,
 8226,
 8265,
 8266,
 8267,
 8268,
 8269,
 8270,
 8271,
 8272,
 8273,
 8274,
 8275,
 8276,
 8277,
 8289,
 8290,
 8291,
 8292,
 8293,
 8294,
 8295,
 8296,
 8297,
 8298,
 8299,
 8337,
 8338,
 8339,
 8340,
 8341,
 8342,
 8343,
 8344,
 8345,
 8346,
 8347,
 8348,
 8360,
 8361,
 8362,
 8363,
 8364,
 8365,
 8366,
 8367,
 8368,
 8369,
 8370,
 8385,
 8386,
 8387,
 8388,
 8389,
 8390,
 8391,
 8392,
 8

In [180]:
z_fossil_hard_coal=np.abs(stats.zscore(cleaned_data['fossil_hard_coal']))
print(z_fossil_hard_coal)

[1.44145167 1.29150476 1.25061014 ... 1.3460309  1.3460309  1.29150476]


In [181]:
threshold_fhc = 3
tup_fhc = np.where(z_fossil_hard_coal > 3)

In [182]:
lis_fhc = list(tup_fhc[0])
lis_fhc

[1378,
 1505,
 1506,
 1507,
 1508,
 1509,
 1531,
 1545,
 1578,
 1579,
 1580,
 1640,
 1645,
 1646]

In [183]:
len(lis_fhc) #14 outliers detected in fossil_hard_coal

14

In [184]:
z_fossil_oil=np.abs(stats.zscore(cleaned_data['fossil_oil']))
print(z_fossil_oil)


[0.78831332 0.78831332 0.78831332 ... 0.45995542 0.45995542 0.45995542]


In [185]:
threshold_fo = 3
tup_fo = np.where(z_fossil_oil > 3)

In [186]:
lis_fo = list(tup_fo[0])
lis_fo

[228,
 1210,
 1211,
 1351,
 1352,
 1353,
 1378,
 1499,
 1811,
 1812,
 1813,
 2005,
 2006,
 2007,
 3051,
 3052,
 3671,
 3731,
 3732,
 3733,
 3734,
 3739,
 3740,
 3960,
 3963,
 4026,
 4027,
 4028,
 4041,
 4042,
 4043,
 4044,
 4045,
 4046,
 4183,
 4195,
 4208,
 4209,
 4210,
 4211,
 4212,
 4213,
 4214,
 4268,
 4337,
 4338,
 4339,
 4340,
 4341,
 4920,
 5001,
 5219,
 5245,
 5246,
 5247,
 5248,
 5249,
 5250,
 5251,
 5252,
 5433,
 5442,
 5443,
 5509,
 5512,
 5513,
 5514,
 5537,
 5538,
 5539,
 5586,
 5587,
 5608,
 6202,
 6203,
 6210,
 6211,
 6212,
 6427,
 6428,
 7176,
 7595,
 7596,
 7597,
 7622,
 7698,
 7699,
 7762,
 7857,
 7906,
 7907,
 7908,
 7915,
 8556,
 8557,
 8558]

In [187]:
len(lis_fo) #96 outliers detected in fossil_oil

96

In [188]:
z_solar=np.abs(stats.zscore(cleaned_data['solar']))
print(z_solar)

[0.61748875 0.61748875 0.61748875 ... 0.61748875 0.61748875 0.61748875]


In [189]:
threshold_so = 3
tup_so = np.where(z_solar > 3)

In [190]:
lis_so = list(tup_so[0])
lis_so

[2316,
 2317,
 2318,
 2340,
 2341,
 2342,
 2388,
 2389,
 2390,
 2461,
 2508,
 2509,
 2510,
 2533,
 2534,
 2557,
 2558,
 2652,
 2653,
 2654,
 2676,
 2677,
 2678,
 2700,
 2701,
 2702,
 2724,
 2725,
 2726,
 2988,
 2989,
 2990,
 3059,
 3060,
 3061,
 3062,
 3109,
 3110,
 3156,
 3157,
 3158,
 3228,
 3229,
 3230,
 3349,
 3372,
 3373,
 3374,
 3563,
 3564,
 3588,
 3636,
 3637,
 3638,
 5268,
 5269,
 5270,
 5271,
 5294,
 5341,
 5365,
 5366,
 5367,
 5389,
 5390,
 5413,
 5414,
 5415,
 5436,
 5437,
 5438,
 5439,
 5844,
 5845,
 5846,
 5847,
 5867,
 5868,
 5869,
 6253,
 6254,
 6276,
 6277,
 6278]

In [191]:
len(lis_so) #84 outliers detected in solar

84

In [192]:
z_waste=np.abs(stats.zscore(cleaned_data['waste']))
print(z_waste)

[0.87510842 0.94994734 0.87510842 ... 1.54865865 0.72543059 0.57575276]


In [193]:
threshold_wa = 3
tup_wa = np.where(z_waste > 3)

In [194]:
lis_wa = list(tup_wa[0])
lis_wa

[1378, 3921, 6056, 6057, 6106, 8082, 8083]

In [195]:
len(lis_wa) #7 outliers detected in waste


7

In [196]:
z_wind_offshore =np.abs(stats.zscore(cleaned_data['wind_offshore']))
print(z_wind_offshore)

[0.24207622 0.35532228 0.42812332 ... 0.31606506 0.52637917 0.68007024]


In [197]:
threshold_wno = 3
tup_wno = np.where(z_wind_offshore > 3)

In [198]:
lis_wno = list(tup_wno[0])
lis_wno

[]

In [199]:
len(lis_wno) #no outliers detected in Wind_offshore

0

In [200]:
z_wind_onshore =np.abs(stats.zscore(cleaned_data['wind_onshore']))
print(z_wind_onshore)

[0.16508308 0.03485172 0.00402449 ... 0.21257154 0.07928168 0.03179321]


In [201]:
threshold_wnon = 3
tup_wnon = np.where(z_wind_onshore > 3)

In [202]:
lis_wnon = list(tup_wnon[0])
lis_wnon

[]

In [203]:
len(lis_wnon) #no outliers detected in Wind_onshore

0

# Outlier analysis
Removing the rows that carries outliers and seeing if that affetcs the data in any way. 
Columns that has outliers are: fossil_gas,fossil_hard_coal,fossil_oil,solar,waste

In [204]:
cleaned_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1,January
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2,January
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3,January
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4,January
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5,January
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6,January
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7,January
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8,January
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9,January
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10,January


In [205]:
test_data=pd.DataFrame(cleaned_data)
test_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1,January
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2,January
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3,January
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4,January
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5,January
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6,January
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7,January
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8,January
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9,January
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10,January


In [206]:
test_data['z_fossil_gas'] = z_fossil_gas
test_data

,date_hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month,z_fossil_gas
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01,Wednesday,1,1,January,0.260333
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01,Wednesday,1,2,January,0.260333
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01,Wednesday,1,3,January,0.469104
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01,Wednesday,1,4,January,0.528753
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01,Wednesday,1,5,January,0.528753
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01,Wednesday,1,6,January,0.528753
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01,Wednesday,1,7,January,0.528753
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01,Wednesday,1,8,January,0.498929
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01,Wednesday,1,9,January,0.469104
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01,Wednesday,1,10,January,0.469104


In [207]:
test_data = test_data.drop(test_data[test_data.z_fossil_gas > 3].index)
print(cleaned_data.shape, test_data.shape, len(lis_fg))

(8785, 15) (8617, 15) 168


In [208]:
print(test_data['fossil_gas'].mean(), cleaned_data['fossil_gas'].mean())

47.12103980503655 49.728856004553215


In [209]:
cleaned_data.describe() #looking at the std removing outliers does not affect the data 

,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,week_number,hour,z_fossil_gas
count,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000
mean,201.600000,49.728856,92.256118,16.630507,45.605350,60.306773,148.926579,225.275356,26.930791,11.499032,0.673680
std,161.935726,33.531447,73.363473,16.023103,73.860368,13.362793,123.631650,180.068950,15.099209,6.922781,0.739065
min,1.000000,11.000000,17.000000,4.000000,0.000000,15.000000,0.000000,13.000000,1.000000,0.000000,0.008087
25%,18.000000,29.000000,45.000000,9.000000,0.000000,52.000000,38.000000,73.000000,14.000000,5.000000,0.200684
50%,194.000000,43.000000,70.000000,14.000000,1.000000,60.000000,119.000000,171.000000,27.000000,11.000000,0.528753
75%,316.000000,57.000000,128.000000,21.000000,64.000000,71.000000,256.000000,356.000000,40.000000,17.000000,0.856822
max,662.000000,889.000000,3599.000000,304.000000,311.000000,146.000000,377.000000,724.000000,53.000000,23.000000,25.030799


In [210]:
test2_data=pd.DataFrame(cleaned_data)

In [211]:
test2_data['z_fossil_oil'] = z_fossil_oil

In [212]:
test2_data = test2_data.drop(test2_data[test2_data.z_fossil_oil > 3].index)
print(cleaned_data.shape, test2_data.shape, len(lis_fo))

(8785, 16) (8689, 16) 96


In [213]:
print(test2_data['fossil_oil'].mean(), cleaned_data['fossil_oil'].mean()) #very minimal impact on the mean

15.38036597997468 16.63050654524758


#### From the above analysis it is evident that the presence of the outliers in this dataset does not affect the overall dataset. Hence we are not going to remove the outliers. 


# Importing the cleaned data to the database

In [214]:
cleaned_data.dtypes

date_hour                   object
biomass                    float64
fossil_gas                 float64
fossil_hard_coal           float64
fossil_oil                 float64
solar                      float64
waste                      float64
wind_offshore              float64
wind_onshore               float64
date                datetime64[ns]
weekday                     object
week_number                  int64
hour                         int64
month                       object
z_fossil_gas               float64
z_fossil_oil               float64
dtype: object

In [215]:
del cleaned_data['z_fossil_gas']

In [216]:
del cleaned_data['z_fossil_oil']

In [218]:
c.execute( 
    """ 
    CREATE TABLE cleaned_data(
                    date_Hour TEXT,
                    biomass REAL,
                    fossil_gas REAL,
                    fossil_hard_coal REAL,
                    fossil_oil REAL,
                    solar REAL,
                    waste REAL,
                    wind_offshore REAL,
                    wind_onshore REAL,
                    date TEXT,
                    weekday TEXT,
                    week_number INT,
                    hour INT,
                    month TEXT)
                    
                    ;
"""
)

In [219]:
#importing the cleaned_data into the database
cleaned_data.to_sql('cleaned_data', conn, if_exists='append', index=False)

In [220]:
pd.read_sql("SELECT * FROM cleaned_data LIMIT 100", conn)

,date_Hour,biomass,fossil_gas,fossil_hard_coal,fossil_oil,solar,waste,wind_offshore,wind_onshore,date,weekday,week_number,hour,month
0,01.01.2020 00:00 - 01.01.2020 01:00 (CET),432.0,41.0,198.0,4.0,0.0,72.0,119.0,255.0,2020-01-01 00:00:00,Wednesday,1,1,January
1,01.01.2020 01:00 - 01.01.2020 02:00 (CET),460.0,41.0,187.0,4.0,0.0,73.0,105.0,219.0,2020-01-01 00:00:00,Wednesday,1,2,January
2,01.01.2020 02:00 - 01.01.2020 03:00 (CET),413.0,34.0,184.0,4.0,0.0,72.0,96.0,226.0,2020-01-01 00:00:00,Wednesday,1,3,January
3,01.01.2020 03:00 - 01.01.2020 04:00 (CET),411.0,32.0,180.0,4.0,0.0,73.0,114.0,239.0,2020-01-01 00:00:00,Wednesday,1,4,January
4,01.01.2020 04:00 - 01.01.2020 05:00 (CET),458.0,32.0,184.0,8.0,0.0,70.0,137.0,254.0,2020-01-01 00:00:00,Wednesday,1,5,January
5,01.01.2020 05:00 - 01.01.2020 06:00 (CET),409.0,32.0,186.0,12.0,0.0,64.0,144.0,302.0,2020-01-01 00:00:00,Wednesday,1,6,January
6,01.01.2020 06:00 - 01.01.2020 07:00 (CET),410.0,32.0,185.0,12.0,0.0,63.0,168.0,329.0,2020-01-01 00:00:00,Wednesday,1,7,January
7,01.01.2020 07:00 - 01.01.2020 08:00 (CET),460.0,33.0,188.0,12.0,0.0,64.0,168.0,346.0,2020-01-01 00:00:00,Wednesday,1,8,January
8,01.01.2020 08:00 - 01.01.2020 09:00 (CET),479.0,34.0,187.0,12.0,0.0,64.0,149.0,309.0,2020-01-01 00:00:00,Wednesday,1,9,January
9,01.01.2020 09:00 - 01.01.2020 10:00 (CET),456.0,34.0,165.0,13.0,6.0,65.0,130.0,316.0,2020-01-01 00:00:00,Wednesday,1,10,January


In [221]:
conn.close()